In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!unzip "/content/drive/MyDrive/Dataset/train.ft.txt.bz2.zip" -d "/content/Amazon Reviews"

Archive:  /content/drive/MyDrive/Dataset/train.ft.txt.bz2.zip
  inflating: /content/Amazon Reviews/train.ft.txt.bz2  


In [4]:
import bz2 as bz2
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('/content/Amazon Reviews/train.ft.txt.bz2')

In [6]:
from sklearn.model_selection import train_test_split
labels_train,discard_labels,texts_train,discard_texts = train_test_split(train_labels,train_texts,
                                                 test_size = 0.99,
                                                 random_state = 2021)

In [51]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_object = PorterStemmer()
from nltk.stem.snowball import SnowballStemmer
snowball_object = SnowballStemmer("english")

In [52]:
reviews_train = []
for text in texts_train:
    text = text.lower() #Convert to lower case
    text = re.sub('[^a-zA-Z ]','',text)
    text = text.split()
    text = [porter_object.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    reviews_train.append(text)

In [135]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVec = TfidfVectorizer()
x = tfidfVec.fit_transform(reviews_train)

In [106]:
y=labels_train


In [107]:
y = np.reshape(y,(-1,1))
print(y.shape)
print(x.shape)

(36000, 1)
(36000, 71990)


In [54]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,
                                                 test_size = 0.2,
                                                 random_state = 2021)

<h1>random forest classification</h1>
criterion='gini

In [136]:
from sklearn.ensemble import RandomForestClassifier
nb_classifier = RandomForestClassifier()
nb_classifier.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [212]:
import pickle
pickle.dump(nb_classifier,open('ASA_NB_rf_cg.sav','wb'))

In [213]:
nb_loaded_classifier = pickle.load(open('/content/ASA_NB_rf_cg.sav','rb'))

In [137]:
y_pred = nb_classifier.predict(x_test)

In [138]:
from sklearn.metrics import accuracy_score
print("Accuracy: ",accuracy_score(y_test,y_pred)*100)

Accuracy:  84.77777777777777


In [139]:
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(y_test,y_pred)
print(c_m)

[[3092  491]
 [ 605 3012]]


In [140]:
from sklearn.metrics import precision_score
prec = precision_score(y_test,y_pred,pos_label = 0)
print("Out of all the data predicted as negative , what is the accuracy?")
print(prec*100)
from sklearn.metrics import precision_score
prec = precision_score(y_test,y_pred,pos_label = 1)
print("Out of all the data predicted as Positive, what is the accuracy?")
print(prec*100)

Out of all the data predicted as negative , what is the accuracy?
83.63538003786854
Out of all the data predicted as Positive, what is the accuracy?
85.9834427633457


In [141]:
from sklearn.metrics import f1_score 
f1 = f1_score(y_test,y_pred)
print(f1*100)

84.60674157303372


<h1>random forest classification</h1>
criterion='entropy'

In [206]:
from sklearn.ensemble import RandomForestClassifier
nb_classifier2 = RandomForestClassifier(criterion='entropy')
nb_classifier2.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [228]:
import pickle
pickle.dump(nb_classifier2,open('ASA_NB_rf_ce.sav','wb'))

In [215]:
nb_loaded_classifier = pickle.load(open('/content/ASA_NB_rf_ce.sav','rb'))

In [207]:
y_pred2 = nb_classifier2.predict(x_test)

In [208]:
print("Accuracy: ",accuracy_score(y_test,y_pred2)*100)

Accuracy:  84.95833333333334


In [209]:
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(y_test,y_pred2)
print(c_m)

[[3125  458]
 [ 625 2992]]


In [210]:
from sklearn.metrics import precision_score
prec = precision_score(y_test,y_pred2,pos_label = 0)
print("Out of all the data predicted as negative , what is the accuracy?")
print(prec*100)
from sklearn.metrics import precision_score
prec = precision_score(y_test,y_pred2,pos_label = 1)
print("Out of all the data predicted as Positive, what is the accuracy?")
print(prec*100)

Out of all the data predicted as negative , what is the accuracy?
83.33333333333334
Out of all the data predicted as Positive, what is the accuracy?
86.72463768115942


In [211]:
from sklearn.metrics import f1_score 
f1 = f1_score(y_test,y_pred2)
print(f1*100)

84.67525116739778


<h1>Decision tree classification</h1>
criterion="entropy"

In [165]:
from sklearn.tree import DecisionTreeClassifier
dt_classifer=DecisionTreeClassifier(criterion='entropy')
dt_classifer.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [216]:
import pickle
pickle.dump(nb_classifier,open('ASA_NB_dt_ce.sav','wb'))

In [167]:
nb_dt_loaded_classifier = pickle.load(open('/content/ASA_NB_dt.sav_ce','rb'))

In [170]:
z_pred=dt_classifer.predict(x_test)

In [171]:
print("Accuracy: ",accuracy_score(y_test,z_pred)*100)

Accuracy:  73.59722222222223


In [173]:
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(y_test,z_pred)
print(c_m)

[[2621  962]
 [ 939 2678]]


In [176]:
from sklearn.metrics import precision_score
prec = precision_score(y_test,z_pred,pos_label = 0)
print("Out of all the data predicted as Negative, what is the accuracy?")
print(prec*100)
from sklearn.metrics import precision_score
prec = precision_score(y_test,z_pred,pos_label = 1)
print("Out of all the data predicted as Positive, what is the accuracy?")
print(prec*100)

Out of all the data predicted as NOT PURCHASING, what is the accuracy?
73.62359550561798
Out of all the data predicted as PURCHASING, what is the accuracy?
73.57142857142858


In [177]:
from sklearn.metrics import f1_score 
f1 = f1_score(y_test,z_pred)
print(f1*100)

73.80460245280418


criterion="gini'

In [178]:
from sklearn.tree import DecisionTreeClassifier
dt_classifer2=DecisionTreeClassifier()
dt_classifer2.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [225]:
import pickle
pickle.dump(nb_classifier2,open('ASA_NB_dt_cg.sav','wb'))

In [182]:
z_pred2=dt_classifer2.predict(x_test)

In [183]:
print("Accuracy: ",accuracy_score(y_test,z_pred2)*100)

Accuracy:  73.61111111111111


In [184]:
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(y_test,z_pred2)
print(c_m)

[[2599  984]
 [ 916 2701]]


In [186]:
from sklearn.metrics import precision_score
prec = precision_score(y_test,z_pred2,pos_label = 0)
print("Out of all the data predicted as Negative, what is the accuracy?")
print(prec*100)
from sklearn.metrics import precision_score
prec = precision_score(y_test,z_pred2,pos_label = 1)
print("Out of all the data predicted as Positive, what is the accuracy?")
print(prec*100)

Out of all the data predicted as Negative, what is the accuracy?
73.9402560455192
Out of all the data predicted as Positive, what is the accuracy?
73.29715061058344


In [187]:
from sklearn.metrics import f1_score 
f1 = f1_score(y_test,z_pred)
print(f1*100)

73.80460245280418


<h1>Mutinomial classification</h1>


In [198]:
from sklearn.naive_bayes import MultinomialNB
Mn_classifier = MultinomialNB()
Mn_classifier.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [192]:
import pickle
pickle.dump(Mn_classifier,open('ASA_NB_Mn.sav','wb'))

In [194]:
nb_Mn_loaded_classifier = pickle.load(open('ASA_NB_Mn.sav','rb'))

In [200]:
Mn_pred= Mn_classifier.predict(x_test)

In [202]:
from sklearn.metrics import accuracy_score
print("Accuracy: ",accuracy_score(y_test,Mn_pred)*100)

Accuracy:  82.06944444444444


In [201]:
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(y_test,Mn_pred)
print(c_m)

[[3097  486]
 [ 805 2812]]


In [203]:
from sklearn.metrics import precision_score
prec = precision_score(y_test,Mn_pred,pos_label = 0)
print("Out of all the data predicted as Negative, what is the accuracy?")
print(prec*100)
from sklearn.metrics import precision_score
prec = precision_score(y_test,Mn_pred,pos_label = 1)
print("Out of all the data predicted as Positive, what is the accuracy?")
print(prec*100)

Out of all the data predicted as Negative, what is the accuracy?
79.36955407483342
Out of all the data predicted as Positive, what is the accuracy?
85.26379624014554


In [204]:
from sklearn.metrics import f1_score 
f1 = f1_score(y_test,Mn_pred)
print(f1*100)

81.33044107013738
